# 爬蟲練習

## 郭耀仁

## 批踢踢實業坊

- 熱門看板
- 一般看板
- 十八禁看板

## 熱門看板

- 網址：<https://www.ptt.cc/bbs/hotboards.html>
- 先練習將第一欄：版名抓出來

In [1]:
library(rvest)

url <- "https://www.ptt.cc/bbs/hotboards.html"
raw_html <- read_html(url)
board_css <- ".board-name"
boards <- raw_html %>%
    html_nodes(css = board_css) %>%
    html_text()
head(boards)
length(boards)

Loading required package: xml2


[1] "Gossiping" "NBA"       "LoL"       "C_Chat"    "Baseball"  "Stock"

[1] 128

## 熱門看板（2）

- 練習將第二欄：人氣抓出來

In [ ]:
library(rvest)

url <- "https://www.ptt.cc/bbs/hotboards.html"
raw_html <- read_html(url)
viewer_css <- "___"
viewers <- raw_html %>%
    html_nodes(css = viewer_css) %>%
    html_text()
head(viewers)
length(viewers)

## 熱門看板（3）

- 是否覺得跟第一個欄位的程式很像？
- 我們只需要更換 CSS Selector 或 XPath 而已
- 寫在 for 迴圈裡面，然後用一個函數包起來：

In [6]:
library(rvest)

ptt_hot_boards_crawler <- function(){
  url <- "https://www.ptt.cc/bbs/hotboards.html"
  raw_html <- read_html(url)
  
  board_xpath <- "//a[@class='board']/div[@class='board-name']"
  viewers_xpath <- "//a[@class='board']/div[@class='board-nuser']/span"
  category_xpath <- "//a[@class='board']/div[@class='board-class']"
  title_xpath <- "//a[@class='board']/div[@class='board-title']"
  
  hot_boards_columns <- list()
  columns <- c(board_xpath, viewers_xpath, category_xpath, title_xpath)
  
  for (i in 1:length(columns)){
    content <- raw_html %>%
      html_nodes(xpath = columns[i]) %>%
      html_text()
    hot_boards_columns[[i]] <- content
  }
  names(hot_boards_columns) <- c("board", "viewers", "category", "title")
  return(hot_boards_columns)
}

## 熱門看板（4）

- `ptt_hot_boards_cralwer()` 函數不需要輸入，可以直接呼叫
- 輸入是一個元素有命名清單（list）

In [9]:
current_ptt_hot_boards <- ptt_hot_boards_crawler()
head(current_ptt_hot_boards$board)
head(current_ptt_hot_boards$viewers)
head(current_ptt_hot_boards$category)
head(current_ptt_hot_boards$title)

[1] "Gossiping" "NBA"       "LoL"       "C_Chat"    "Baseball"  "Stock"

[1] "12952" "11695" "2830"  "2453"  "2330"  "1700"

[1] "綜合" "籃球" "遊戲" "聊天" "棒球" "學術"

[1] "◎[八卦] 夏天熱肥宅"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
[2] "◎[NBA]總冠軍賽6/8 9:00 詠琪大戰"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
[3] "◎[LoL] LMS本日補賽進行中"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
[4] "◎[希洽] 2016希洽人氣動畫提名中！"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
[5] "◎[棒球] YOH來了   [email protected]/* <![CDATA[ */!function(t,e,r,n,c,a,p){try{t=document.currentScript||function(){for(t=document.getElementsByTagName('script'),e=t.length;e--;)if(t[e].getAttribute('data-cfhash'))return t[e]}();if(t&&(c=t.previousSibling)){p=t.parentNode;if(a=c.getAttribute('data-cfemail')){for(e='',r='0x'+a.substr(0,2)|0,n=2;a.length-n;n+=2)e+='%'+('0'+('0x'+a.substr(n,2)^r).toString(16)).slice(-2);p.replaceChild(document.createTextNode(decodeURIComponent(e)),c)}p.removeChild(t)}}catch(u){}}()/* ]]> */"
[6] "◎[股板] 徵文比賽中 好文連載中"

## 熱門看板（5）

- 練習將熱門看板的連結抓出來
- 提示：
    - css selector 改用 `.board`
    - 改用 `html_attr("href")` 而非 `html_text()`

## 一般看板

- 以 NBA 為例：<https://www.ptt.cc/bbs/NBA/index.html>
- 要怎麼將一整頁的文章內容都抓出來呢？

## 一般看板（2）

- 先把看得到的抓出來：
    - 推文數
    - 文章標題
    - ID

In [11]:
library(rvest)

url <- "https://www.ptt.cc/bbs/NBA/index.html"
raw_html <- read_html(url)

nrec_xpath <- "//div[@class='nrec']"
title_xpath <- "//div[@class='title']"
id_xpath <- "//div[@class='meta']/div[@class='author']"

article_info <- list()
columns <- c(nrec_xpath, title_xpath, id_xpath)

for (i in 1:length(columns)){
  content <- raw_html %>%
    html_nodes(xpath = columns[i]) %>%
    html_text()
  article_info[[i]] <- content
}

names(article_info) <- c("nrec", "title", "author_id")

## 一般看板（3）

- 接著我們要清理一下文章標題

In [14]:
head(article_info$title)
pattern_to_clean <- "\n\t+n\t+"
head(gsub(pattern = "\n\t+\n\t+", article_info$title, replacement = ""))

[1] "\n\t\t\t\n\t\t\t\t[討論] 是不是明年又要再看一次勇騎大戰?\n\t\t\t\n\t\t\t"
[2] "\n\t\t\t\n\t\t\t\t[討論] 請問關於今年勇士的一個小問題\n\t\t\t\n\t\t\t"   
[3] "\n\t\t\t\n\t\t\t\t[討論]騎士最後的犯規時間點\n\t\t\t\n\t\t\t"            
[4] "\n\t\t\t\n\t\t\t\t[公告] 板規6.1\n\t\t\t\n\t\t\t"                        
[5] "\n\t\t\t\n\t\t\t\t[公告] 違規檢舉區\n\t\t\t\n\t\t\t"                     
[6] "\n\t\t\t\n\t\t\t\t[情報] 2016-2017 總冠軍賽 (6/2 - 6/19)\n\t\t\t\n\t\t\t"

[1] "[討論] 是不是明年又要再看一次勇騎大戰?"
[2] "[討論] 請問關於今年勇士的一個小問題"   
[3] "[討論]騎士最後的犯規時間點"            
[4] "[公告] 板規6.1"                        
[5] "[公告] 違規檢舉區"                     
[6] "[情報] 2016-2017 總冠軍賽 (6/2 - 6/19)"

## 一般看板（4）

- 隨便點一篇文章進去看
- 我們要抓文章的哪些內容？
    - 作者
    - 標題
    - 時間
    - 內文
    - IP
    - 推文標記
    - 推文 ID
    - 推文內容
    - 推文時間

In [ ]:
library(rvest)

article_detail <- function(url){
    raw_html <- read_html(url)
    
    author_css <- ".article-metaline:nth-child(1) .article-meta-value"
    title_css <- ".article-metaline-right+ .article-metaline .article-meta-value"
    time_css <- ".article-metaline+ .article-metaline .article-meta-value"
    main_content_css <- "#main-content"
    ip_css <- ".hl+ .f2"
    push_css <- ".push-tag"
    push_id_css <- ".push-userid"
    push_content_css <- ".push-content"
    push_time_css <- ".push-ipdatetime"
    
    article_detail_info <- list()
    columns <- c(author_css, title_css, time_css, main_content_css, ip_css, push_css, push_id_css, push_content_css, push_time_css)
    for (i in 1:length(columns)){
        article_content <- raw_html %>%
            html_nodes(css = columns[i]) %>%
            html_text()
        article_detail_info[[i]] <- article_content
    }
    names(article_detail_info) <- c("author", "title", "time", "main_content", "ip", "push", "push_id", "push_content", "push_time")
    return(article_detail_info)
}

In [ ]:
article_url <- "https://www.ptt.cc/bbs/NBA/M.1496931569.A.682.html"
nba_article_details <- article_detail(article_url)

## 一般看板（5）

- 清理內文
- 清理IP
- 清理推文
- 清理推文 ID
- 清理推文內容
- 清理推文時間

## 一般看板（6）

- 清理內文

In [ ]:
nba_article_details$main_content <- nba_article_details$main_content %>%
    gsub(pattern = "\n", ., replacement = "") %>% # 清理斷行符號
    gsub(pattern = "--.+", ., replacement = "") %>% # 去尾
    gsub(pattern = "作者.+:[0-9]{2}\\s[0-9]{4}", ., replacement = "") # 去頭

## 一般看板（7）

- 清理IP

In [ ]:
ip_start <- regexpr(pattern = "[0-9]+", nba_article_details$ip)
ip_end <- nchar(nba_article_details$ip)
nba_article_details$ip <- substr(nba_article_details$ip, start = ip_start, stop = ip_end)
nba_article_details$ip <- gsub(pattern = "\n", nba_article_details$ip, replacement = "") # 清理斷行符號

## 一般看板（8）

- 清理推文
- 清理推文 ID
- 清理推文內容
- 清理推文時間

In [ ]:
nba_article_details$push <- gsub(pattern = "\\s", nba_article_details$push, replacement = "")
nba_article_details$push_id <- gsub(pattern = "\\s", nba_article_details$push_id, replacement = "")
nba_article_details$push_content <- nba_article_details$push_content %>%
    gsub(pattern = "\\s", ., replacement = "") %>%
    gsub(pattern = ":", ., replacement = "")
nba_article_details$push_time <- nba_article_details$push_time %>%
    gsub(pattern = "^\\s", ., replacement = "") %>%
    gsub(pattern = "\n", ., replacement = "")

## 一般看板（8）

- 最後我們把清理資料的程式加入原本的 `article_detail()` 函數中

In [ ]:
library(rvest)

article_detail <- function(url){
    raw_html <- read_html(url)
    
    author_css <- ".article-metaline:nth-child(1) .article-meta-value"
    title_css <- ".article-metaline-right+ .article-metaline .article-meta-value"
    time_css <- ".article-metaline+ .article-metaline .article-meta-value"
    main_content_css <- "#main-content"
    ip_css <- ".hl+ .f2"
    push_css <- ".push-tag"
    push_id_css <- ".push-userid"
    push_content_css <- ".push-content"
    push_time_css <- ".push-ipdatetime"
    
    article_detail_info <- list()
    columns <- c(author_css, title_css, time_css, main_content_css, ip_css, push_css, push_id_css, push_content_css, push_time_css)
    for (i in 1:length(columns)){
        article_content <- raw_html %>%
            html_nodes(css = columns[i]) %>%
            html_text()
        article_detail_info[[i]] <- article_content
    }
    
    names(article_detail_info) <- c("author", "title", "time", "main_content", "ip", "push", "push_id", "push_content", "push_time")
    article_detail_info$main_content <- article_detail_info$main_content %>%
        gsub(pattern = "\n", ., replacement = "") %>% # 清理斷行符號
        gsub(pattern = "--.+", ., replacement = "") %>% # 去尾
        gsub(pattern = "作者.+:[0-9]{2}\\s[0-9]{4}", ., replacement = "") # 去頭
    ip_start <- regexpr(pattern = "[0-9]+", article_detail_info$ip)
    ip_end <- nchar(article_detail_info$ip)
    article_detail_info$ip <- substr(article_detail_info$ip, start = ip_start, stop = ip_end)
    article_detail_info$ip <- gsub(pattern = "\n", article_detail_info$ip, replacement = "") # 清理斷行符號
    article_detail_info$push <- gsub(pattern = "\\s", article_detail_info$push, replacement = "")
    article_detail_info$push_id <- gsub(pattern = "\\s", article_detail_info$push_id, replacement = "")
    article_detail_info$push_content <- article_detail_info$push_content %>%
        gsub(pattern = "\\s", ., replacement = "") %>%
        gsub(pattern = ":", ., replacement = "")
    article_detail_info$push_time <- article_detail_info$push_time %>%
        gsub(pattern = "^\\s", ., replacement = "") %>%
        gsub(pattern = "\n", ., replacement = "")
    return(article_detail_info)
}

## 一般看板（9）

- 接著我們回到最新一頁將所有文章連結抓出來

In [17]:
library(rvest)

url <- "https://www.ptt.cc/bbs/NBA/index.html"
raw_html <- read_html(url)

article_link_xpath <- "//div[@class='title']/a"
article_links <- raw_html %>%
    html_nodes(xpath = article_link_xpath) %>%
    html_attr("href")
article_links # relative url
article_links <- paste("https://www.ptt.cc", article_links, "")
article_links # absolute url

[1] "/bbs/NBA/M.1496913784.A.819.html" "/bbs/NBA/M.1496913913.A.D13.html"
[3] "/bbs/NBA/M.1496914207.A.319.html" "/bbs/NBA/M.1496914864.A.E3F.html"
[5] "/bbs/NBA/M.1480648734.A.D28.html" "/bbs/NBA/M.1478497499.A.150.html"
[7] "/bbs/NBA/M.1486126665.A.D3C.html" "/bbs/NBA/M.1496893864.A.9E3.html"

[1] "https://www.ptt.cc /bbs/NBA/M.1496913784.A.819.html"
[2] "https://www.ptt.cc /bbs/NBA/M.1496913913.A.D13.html"
[3] "https://www.ptt.cc /bbs/NBA/M.1496914207.A.319.html"
[4] "https://www.ptt.cc /bbs/NBA/M.1496914864.A.E3F.html"
[5] "https://www.ptt.cc /bbs/NBA/M.1480648734.A.D28.html"
[6] "https://www.ptt.cc /bbs/NBA/M.1478497499.A.150.html"
[7] "https://www.ptt.cc /bbs/NBA/M.1486126665.A.D3C.html"
[8] "https://www.ptt.cc /bbs/NBA/M.1496893864.A.9E3.html"

## 一般看板（10）

- 一篇文章的詳細內容存在一個清單中
- 外面再用一個清單儲存這些清單（Nested lists）

In [ ]:
article_lists <- list()
for (i in 1:length(article_links)){
    article_details[[i]] <- article_detail(article_links[i])
}